### EDA 2022


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
spark = SparkSession.builder.appName('2022EDA').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','4g')])

#print spark configuration settings
spark.sparkContext.getConf().getAll()

In [ ]:
df_2022 = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/rideshare/2022", inferSchema=True, header=True)
# figure out how to read in shp file msca-bdp-student-gcs/bdp-rideshare-project/neighborhoods/shp files
#df_weather = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/weather/chicago 2020-01-01 to 2022-08-31.csv", inferSchema=True, header=True)
df_2022.printSchema()
#df_weather.printSchema()

In [ ]:
#display number of records by partition
def displaypartitions(df):
    #number of records by partition
    num = df.rdd.getNumPartitions()
    print("Partitions:", num)
    df.withColumn("partitionId", F.spark_partition_id())\
        .groupBy("partitionId")\
        .count()\
        .orderBy(F.asc("count"))\
        .show(num)

df_2022.rdd.getNumPartitions()

In [ ]:
displaypartitions(df_2022)

In [ ]:
# df_2022 = df_2022.repartition(10)

In [ ]:
df_2022.describe().show()

In [ ]:
#Find the number of missing values for each column
from pyspark.sql.functions import isnan, when, count, col
df_2022.select([count(when(df_2022[c].isNull(), c)).alias(c) for c in df_2022.columns]).show()

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df_2022.sample(fraction=1/10000).toPandas())

In [ ]:
#Approximate number of 2022 trips
from pyspark.sql.functions import approxCountDistinct

df_2022.select(approxCountDistinct("Trip ID", rsd = 0.01)).show()

In [ ]:
# number of observations with all the data in each column
df_2022.dropna(how='any').count()

In [ ]:
# Working with just data that contains full information and check for dupes
df_2022 = df_2022.dropna(how='any', subset=['Trip Start Timestamp','Trip End Timestamp','Fare','Dropoff Community Area','Pickup Community Area'])
df_2022 = df_2022.dropDuplicates()
# df_2022.count()

In [ ]:
df_2022 = df_2022.drop('Trips Pooled','Additional Charges','Shared Trip Authorized','Pickup Centroid Location','Dropoff Centroid Location')
df_2022 = df_2022.withColumnRenamed("Trip ID","ID").withColumnRenamed("Trip Start Timestamp","start_timestamp").withColumnRenamed("Trip End Timestamp","end_timestamp").withColumnRenamed("Trip Miles",\
                "miles").withColumnRenamed("Pickup Census Tract","pickup_tract").withColumnRenamed("Dropoff Census Tract","dropoff_tract").withColumnRenamed("Pickup Community Area","pickup_area"\
                ).withColumnRenamed("Dropoff Community Area","dropoff_area").withColumnRenamed("Trip Total","total").withColumnRenamed("Pickup Centroid Latitude","pickup_lat").withColumnRenamed(\
                "Pickup Centroid Longitude","pickup_lon").withColumnRenamed("Pickup Centroid Location","pickup_location").withColumnRenamed("Dropoff Centroid Latitude","dropoff_lat").withColumnRenamed(\
                "Dropoff Centroid Longitude","dropoff_lon").withColumnRenamed("Dropoff Centroid Location","dropoff_location").withColumnRenamed("Trip Seconds","seconds")
# fix datatypes
df_2022 = df_2022.withColumn('start_timestamp', F.to_timestamp(df_2022['start_timestamp'], 'MM/dd/yyyy hh:mm:ss a')).withColumn('end_timestamp', F.to_timestamp(df_2022['end_timestamp'], 'MM/dd/yyyy hh:mm:ss a'))
df_weather = df_weather.withColumn('datetime',F.to_date(df_weather['datetime'], "yyyy-MM-dd"))


In [ ]:
# add the month column
df_2022 = df_2022.withColumn('month', F.month(df_2022.start_timestamp))
df_2022 = df_2022.withColumn('day_of_month', F.dayofmonth(df_2022.start_timestamp))
df_2022 = df_2022.withColumn('hour', F.hour(df_2022.start_timestamp))
df_2022 = df_2022.withColumn('day', F.dayofweek(df_2022.start_timestamp))

In [ ]:
sample_df = df_2022.sample(fraction=1/10000).toPandas().loc[:,["pickup_area","dropoff_area","total","Fare","Tip","total","miles","seconds","hour","day","month"]]

In [ ]:
sample_df.head()

In [ ]:
sample_df = sample_df.dropna()
sample_df = sample_df.drop_duplicates()

In [ ]:
sample_df.head()
sample_df = sample_df.drop(columns='total')

In [ ]:
#import seaborn as sns
#sns.set_theme(style="ticks")
#sns.pairplot(sample_df)
#plt.show()

In [ ]:
import seaborn as sns
sns.displot(sample_df, x="miles")

In [ ]:
sns.displot(sample_df, x="seconds")

In [ ]:
sns.displot(sample_df, x="Fare")

In [ ]:
sns.displot(sample_df, x="Tip")

In [ ]:
# get rides that occurred within hyde park
# add kenwood and woodlawn to this list - only if the other location is hyde park                    
df_hp = df_2022.filter((df_2022.pickup_area == 41) & (df_2022.dropoff_area == 41))
df_kw = df_2022.filter(((df_2022.pickup_area == 41) & (df_2022.dropoff_area == 42)) | ((df_2022.pickup_area == 42) & (df_2022.dropoff_area == 41)))
df_wl = df_2022.filter(((df_2022.pickup_area == 41) & (df_2022.dropoff_area == 39)) | ((df_2022.pickup_area == 39) & (df_2022.dropoff_area == 41)))
df_area = df_hp.union(df_kw).union(df_wl)

In [ ]:
df_area.show(5)

In [ ]:
import matplotlib.pyplot as plt
df_count_plot = df_area.groupby("month").agg({'ID':'count'}).orderBy(F.col('month').asc()).toPandas() #.plot(x="month",y="count(ID)")
ax = df_count_plot.plot(x="month", y="count(ID)")
ax.axvline(x=9, color='g', linestyle='--', label='Program Starts, Fall Quarter Begins')
ax.axvline(x=6, color='r', linestyle='--', label='Spring Quarter Ends')
plt.legend()
plt.show()

In [ ]:
# rides per hour:
rides_per_hour = df_area.groupBy("hour").count().orderBy("hour")

# Convert the result to Pandas for local plotting
rides_per_hour_pd = rides_per_hour.toPandas()

# Plot the data
plt.figure(figsize=(10, 6))
plt.bar(rides_per_hour_pd["hour"], rides_per_hour_pd["count"], color="skyblue")
plt.xlabel("Hour of the Day")
plt.ylabel("Number of Rides")
plt.title("Number of Rides Per Hour")
plt.show()

In [ ]:
df_area_program = df_area.filter((df_area.fare <= 15.0) & ((df_area.hour >= 17) | (df_area.hour < 4)))

In [ ]:
from pyspark.sql.functions import approxCountDistinct

df_area_program.select(approxCountDistinct("ID", rsd = 0.05)).show()

In [ ]:
df_count_plot_program = df_area_program.groupby("month").agg({'ID':'count'}).orderBy(F.col('month').asc()).toPandas() #.plot(x="month",y="count(ID)")
ax = df_count_plot_program.plot(x="month", y="count(ID)")
ax.axvline(x=9, color='g', linestyle='--', label='Program Starts, Fall Quarter Begins')
ax.axvline(x=6, color='r', linestyle='--', label='Spring Quarter Ends')
plt.legend()
plt.show()

In [ ]:
import seaborn as sns

df_area_program_pd = df_area_program.toPandas()

plt.figure(figsize=(10, 6))
sns.histplot(df_area_program_pd['hour'], bins=24, kde=False, color='skyblue')
plt.title('Distribution of Pickup Hours for Filtered Rides')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Rides')
plt.show()

In [ ]:
# basic plots for all rides (not just in the program area)
df_2022.groupby("month").agg({'ID':'count'}).orderBy(F.col('month').asc()).toPandas().plot(x="month",y="count(ID)")

In [ ]:
df_total = df_2022.groupby("pickup_area").agg({'ID':'count'}).orderBy(F.col('pickup_area').asc()).toPandas()

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 10))
ax = df_total.plot(x="pickup_area",y="count(ID)", kind='bar')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.show()

In [ ]:
df_2022.groupby("dropoff_area").agg({'ID':'count'}).orderBy(F.col('dropoff_area').asc()).toPandas().plot(x="dropoff_area",y="count(ID)", kind='bar')

In [ ]:
# storing data on the bucket
df_area_program.write.option("header", "true").csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/rideshare/processed_data/program_area_time_rides_2022.csv")
df_2022.write.option("header", "true").csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/rideshare/processed_data/rides_2022.csv")